In [4]:
import pdfplumber
import re
import os
import pandas as pd

df = pd.read_excel("jurisprudencia_extraida.xlsx")
arquivos_lidos = df['ARQUIVO'].unique()
arquivos_totais = os.listdir('data')

# organizar os dados
resultados = []
for file in os.listdir('data'):

    if file in arquivos_lidos:
        continue
    else:
        
        print(f"Lendo arquivo: {file}")
        # caminho do PDF
        pdf_path = f"data/{file}"

        # regex para capturar o bloco
        # - PROCESSO (até a quebra de linha seguinte)
        # - RAMO DO DIREITO (até a quebra de linha seguinte)
        # - TEMA (até a quebra de linha seguinte)
        # - DESTAQUE (texto abaixo, até antes de PROCESSO seguinte ou fim)
        pattern = re.compile(
            r"PROCESSO\s+(.*?)\s+RAMO DO DIREITO\s+(.*?)\s+TEMA\s+(.*?)\s+DESTAQUE\s+(.*?)(?=INFORMAÇÕES DO INTEIRO TEOR)",
            re.DOTALL
        )

        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                all_text += page.extract_text() + "\n"

        # aplicar regex
        matches = pattern.findall(all_text)

        for processo, ramo, tema, destaque in matches:
            # limpar espaços e quebras de linha
            dados = {
                "PROCESSO": " ".join(processo.split()),
                "RAMO_DO_DIREITO": " ".join(ramo.split()),
                "TEMA": " ".join(tema.split()),
                "DESTAQUE": " ".join(destaque.split()),
                'ARQUIVO': file
            }
            resultados.append(dados)

        # imprimir ou salvar
        #    print("="*80)
        #for r in resultados:
        #    print("PROCESSO:", r["PROCESSO"])
        #    print("RAMO DO DIREITO:", r["RAMO_DO_DIREITO"])
        #    print("TEMA:", r["TEMA"])
        #    print("DESTAQUE:", r["DESTAQUE"])


Lendo arquivo: Inf0639.pdf
Lendo arquivo: Inf0640.pdf
Lendo arquivo: Inf0641.pdf
Lendo arquivo: Inf0642.pdf
Lendo arquivo: Inf0643.pdf
Lendo arquivo: Inf0644.pdf
Lendo arquivo: Inf0645.pdf
Lendo arquivo: Inf0646.pdf
Lendo arquivo: Inf0647.pdf
Lendo arquivo: Inf0648.pdf
Lendo arquivo: Inf0649.pdf
Lendo arquivo: Inf0650.pdf
Lendo arquivo: Inf0651.pdf
Lendo arquivo: Inf0652.pdf
Lendo arquivo: Inf0653.pdf
Lendo arquivo: Inf0654.pdf
Lendo arquivo: Inf0655.pdf
Lendo arquivo: Inf0656.pdf
Lendo arquivo: Inf0657.pdf
Lendo arquivo: Inf0658.pdf
Lendo arquivo: Inf0659.pdf
Lendo arquivo: Inf0660.pdf
Lendo arquivo: Inf0661.pdf
Lendo arquivo: Inf0662.pdf
Lendo arquivo: Inf0663.pdf
Lendo arquivo: Inf0664.pdf
Lendo arquivo: Inf0665.pdf
Lendo arquivo: Inf0666.pdf
Lendo arquivo: Inf0667.pdf
Lendo arquivo: Inf0668.pdf
Lendo arquivo: Inf0669.pdf
Lendo arquivo: Inf0670.pdf
Lendo arquivo: Inf0671.pdf
Lendo arquivo: Inf0672.pdf
Lendo arquivo: Inf0673.pdf
Lendo arquivo: Inf0674.pdf
Lendo arquivo: Inf0675.pdf
L

In [13]:
import pandas as pd

df_ = pd.DataFrame(resultados)

df_['LINK'] = 'https://processo.stj.jus.br/SCON/GetPDFINFJ?edicao=' + df['ARQUIVO'].str.replace('Inf','').str.replace('.pdf','')


In [21]:
df_final = pd.concat([df,df_])

In [23]:
import pandas as pd

df_final.to_excel("jurisprudencia_extraida.xlsx", index=False)

## USO IA

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Resuma o destaque"
    }
]

In [54]:
from groq import Groq

client = Groq()

completion = client.chat.completions.create(
    model="qwen/qwen3-32b",
    messages=messages,
    temperature=0.6,
    max_completion_tokens=4096,
    top_p=0.95,
    reasoning_effort="default",
    stream=True,
    stop=None
)

# Variável para acumular toda a resposta
full_response = ""

for chunk in completion:
    delta = chunk.choices[0].delta.content
    if delta:
        print(delta, end="")
        full_response += delta  # Acumula o texto

# Depois de terminar, adiciona no histórico
messages.append({
    "role": "assistant",
    "content": full_response.split('</think>\n\n')[1]
})

<think>
Okay, the user hasn't sent any messages after my last response. I need to figure out the best way to respond. Since they're not typing anything, maybe they're just browsing or testing the chat. I should keep it friendly and open. Let me offer assistance again in a welcoming manner. Use an emoji to keep it light. Maybe say something like, "How can I assist you today? 😊" That should cover it.
</think>

Como posso ajudar você hoje? 😊

In [55]:
messages

[{'role': 'user', 'content': 'ola'},
 {'role': 'assistant',
  'content': '<think>\nOkay, the user wrote "ola". That\'s Portuguese for "hello". I should respond in Portuguese to be friendly and continue the conversation in their language. Let me make sure I use the correct greeting and maybe ask how they\'re doing. I\'ll keep it simple and welcoming.\n</think>\n\nOlá! Como posso ajudar você hoje? 😊'},
 {'role': 'assistant',
  'content': '<think>\nOkay, the user just sent "ola" which is "hello" in Portuguese. I responded with a friendly greeting. Now they sent an empty message. Maybe they\'re testing if I\'m responsive or just being playful. I should keep the tone light and engaging. Let me ask how I can assist them further to encourage a meaningful conversation. Keeping it open-ended so they can ask anything they need help with.\n</think>\n\nComo posso ajudar você hoje? 😊'},
 {'role': 'assistant',
  'content': '<think>\nOkay, the user just sent an empty message again. Let me check the h